In [2]:
import pandas as pd
from selenium import webdriver
import concurrent.futures

In [8]:
options = webdriver.EdgeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-extensions")
options.add_argument("--disable-infobars")
options.add_argument("--disable-logging")

# open with full screen
options.add_argument("--start-maximized")

options.add_experimental_option(
    "prefs", {
        # block image loading
        "profile.managed_default_content_settings.images": 2,
    }
)

browser1 = webdriver.Edge(options=options)

browser2 = webdriver.Edge(options=options)

browser3 = webdriver.Edge(options=options)

browser1.get('https://www.imdb.com/search/title/?title_type=feature,tv_movie&user_rating=6,10&sort=user_rating,desc&num_votes=50000,')

pagination_btn = browser1.find_element("xpath","//div[contains(@class,'pagination-container')]//button")
for _ in range(300):
    try:
        browser1.execute_script("arguments[0].scrollIntoView();", pagination_btn)
        pagination_btn.click()
        browser1.execute_script("arguments[0].scrollIntoView();", pagination_btn)
    except:
        browser1.implicitly_wait(10)
        

def get_data(xpath, element):
    try:
        return element.find_element("xpath",xpath).text
    except:
        return None

elements = browser1.find_elements("xpath","//ul[contains(@class, 'ipc-metadata-list')]/li")
movies_length = len(elements)
print("Movies length:",movies_length)
movies = []

for element in elements:
    try:
        movies.append({
            "link": element.find_element("xpath",".//a[contains(@class, 'ipc-title-link-wrapper')]").get_attribute("href"),
            "description": get_data(".//div[@class='ipc-html-content-inner-div']",element),
            "rating":get_data(".//span[contains(@class,'ipc-rating-star--imdb')]",element).split(" ")[0].strip(),
            "votes": get_data(".//div[contains(@class,'dli-plot-container')]/following-sibling::div",element).replace("Votes",""),
            "year": get_data(".//div[contains(@class,'dli-title-metadata')]/span[1]",element),
            "duration": get_data(".//div[contains(@class,'dli-title-metadata')]/span[2]",element),
            "certificate": get_data(".//div[contains(@class,'dli-title-metadata')]/span[3]",element),
            "meta_score": get_data(".//span[contains(@class,'metacritic-score-box')]",element),
        })
        print("Movies count:",len(movies))
    except:
        pass



def scrape_movie_data(movies_list,browser):
    for movie in movies_list:
        try:
            browser.get(movie["link"]) 
            movie["title"] = get_data("//h1",browser)
            print(movie["title"])
            movie["type"] = 'Movie'
            movie["popularity"] = get_data("//div[1]/div[1]/div[3]/a[1]/span[1]/div[1]/div[2]/div[1]",browser)
            movie["genre"] = ",".join([genre.text for genre in browser.find_elements("xpath","//div[@data-testid='genres']//a")])
            movie["director"] = ",".join([director.text for director in browser.find_elements("xpath","//div[2]/div/ul/li[1]/div/ul/li/a")])
            movie["writer"] = ",".join([writer.text for writer in browser.find_elements("xpath","//div[2]/div/ul/li[2]/div/ul/li/a")])
            movie["stars"] = ",".join([star.text for star in browser.find_elements("xpath","//div[2]/div/ul/li[3]/div/ul/li/a")])
            movie["country"] = ",".join([country.text for country in browser.find_elements("xpath","//li[@data-testid='title-details-origin']/div/ul/li")])
            movie["language"] = ",".join([language.text for language in browser.find_elements("xpath","//li[@data-testid='title-details-languages']/div/ul/li")])
            movie["production_company"] = ",".join([production_company.text for production_company in browser.find_elements("xpath","//li[@data-testid='title-details-companies']/div/ul/li")])
            movie["worldwide_gross"] = get_data("//li[@data-testid='title-boxoffice-cumulativeworldwidegross']/div//span",browser)
        except:
            pass


scrape_movie_data(movies,browser1)

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.submit(scrape_movie_data,movies[:movies_length//3],browser1)
    executor.submit(scrape_movie_data,movies[movies_length//3:2*(movies_length//3)],browser2)
    executor.submit(scrape_movie_data,movies[2*(movies_length//3):],browser3)

browser1.quit()
browser2.quit()
browser3.quit()

df = pd.DataFrame(movies)
df.to_csv("imdb_movies.csv",index=False)

Free Solo
Citizenfour
Exit Through the Gift Shop
Amy
The Imposter
Fyre
Love Hard
The Saint
A Very Harold & Kumar Christmas
London Boulevard
Step Up 3D
Toofaan
Love
Amsterdam
The Lost City
Ant-Man and the Wasp: Quantumania
Justice League
Jack Reacher: Never Go Back
Knock at the Cabin
Ambulance
Die Another Day
Birds of Prey
Mortal Engines
The Pope's Exorcist
Jurassic World: Fallen Kingdom
Men
Snow White and the Huntsman
The Waterboy
Mission: Impossible II
6 Underground
Bring It On
Ticket to Paradise
San Andreas
Wrong Turn
Vacation
Van Helsing
The Strangers
Sausage Party
The Monuments Men
The Neon Demon
28 Days
Magic Mike
RoboCop
I Am Number Four
Hansel & Gretel: Witch Hunters
Superman Returns
Brightburn
Rampage
A Million Ways to Die in the West
What Happens in Vegas
The Cable Guy
The Expendables 3
The Girl in the Spider's Web
Miami Vice
27 Dresses
Day Shift
Shrek the Third
Never Say Never Again
Gunpowder Milkshake
Hitman's Wife's Bodyguard
Lightyear
S.W.A.T.
Bedazzled
Speed Racer
The Gol

In [7]:
options = webdriver.EdgeOptions()
options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-extensions")
options.add_argument("--disable-infobars")
options.add_argument("--disable-logging")

#open with full screen
options.add_argument("--start-maximized")

options.add_experimental_option(
    "prefs", {
        # block image loading
        "profile.managed_default_content_settings.images": 2,
    }
)

browser1 = webdriver.Edge(options=options)

browser2 = webdriver.Edge(options=options)

browser3 = webdriver.Edge(options=options)

browser1.get('https://www.imdb.com/search/title/?title_type=tv_miniseries,tv_series&user_rating=6,10&sort=user_rating,desc&num_votes=25000,')

pagination_btn = browser1.find_element("xpath","//div[contains(@class,'pagination-container')]//button")
for _ in range(150):
    try:
        browser1.execute_script("arguments[0].scrollIntoView();", pagination_btn)
        pagination_btn.click()
        browser1.execute_script("arguments[0].scrollIntoView();", pagination_btn)
    except:
        browser1.implicitly_wait(10)

def get_data(xpath, element):
    try:
        return element.find_element("xpath",xpath).text
    except:
        return None

elements = browser1.find_elements("xpath","//ul[contains(@class, 'ipc-metadata-list')]/li")
series_length = len(elements)
print("Series length:",series_length)
series = []

for element in elements:
    try:
        series.append({
            "link": element.find_element("xpath",".//a[contains(@class, 'ipc-title-link-wrapper')]").get_attribute("href"),
            "description": get_data(".//div[@class='ipc-html-content-inner-div']",element),
            "rating":get_data(".//span[contains(@class,'ipc-rating-star--imdb')]",element).split(" ")[0].strip(),
            "votes": get_data(".//div[contains(@class,'dli-plot-container')]/following-sibling::div",element).replace("Votes",""),
            "year": get_data(".//div[contains(@class,'dli-title-metadata')]/span[1]",element),
            "certificate": get_data(".//div[contains(@class,'dli-title-metadata')]/span[2]",element),
        })
        print("Series count:",len(series))
    except:
        pass

def scrape_series_data(series_list,browser):
    count = 0
    for series in series_list:
        try:
            if series.get("title"):
                continue
        except:
            pass
        try:
            count += 1
            browser.get(series["link"]) 
            series["title"] = get_data("//h1",browser)
            print(series["title"])
            series["type"] = 'TV Series'
            # series["popularity"] = get_data("//div[1]/div[1]/div[3]/a[1]/span[1]/div[1]/div[2]/div[1]",browser)
            series["genre"] = ",".join([genre.text for genre in browser.find_elements("xpath","//div[@data-testid='genres']//a")])
            try:
                series["creators"] = ",".join([director.text for director in browser.find_elements("xpath","//div[2]/div/ul/li[1]/div/ul/li/a")])
                series["starts"] = ",".join([writer.text for writer in browser.find_elements("xpath","//div[2]/div/ul/li[2]/div/ul/li/a")])
            except:
                series["starts"] = ",".join([director.text for director in browser.find_elements("xpath","//div[2]/div/ul/li[1]/div/ul/li/a")])
            series["country"] = ",".join([country.text for country in browser.find_elements("xpath","//li[@data-testid='title-details-origin']/div/ul/li")])
            series["language"] = ",".join([language.text for language in browser.find_elements("xpath","//li[@data-testid='title-details-languages']/div/ul/li")])
            series["production_company"] = ",".join([production_company.text for production_company in browser.find_elements("xpath","//li[@data-testid='title-details-companies']/div/ul/li")])
            series["duration"] = get_data("//section[@data-testid='TechSpecs']//li[@data-testid='title-techspec_runtime']/div",browser)
            if count>300:
                break
        except:
            pass


scrape_series_data(series,browser1)

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.submit(scrape_movie_data,movies[:movies_length//3],browser1)
    executor.submit(scrape_movie_data,movies[movies_length//3:2*(movies_length//3)],browser2)
    executor.submit(scrape_movie_data,movies[2*(movies_length//3):],browser3)

browser1.quit()
browser2.quit()
browser3.quit()

df = pd.DataFrame(series)
df.to_csv("imdb_series.csv",index=False)

with open("imdb_series.json","w") as f:
    import json
    json.dump(series,f)

3%
Disenchantment
Numb3rs
Dharma & Greg
Dear White People
